In [1]:
import pystan

In [2]:
stan_code = """
## http://openbugs.net/Examples/Surgical.html
## random effects model 
data {
  int<lower=0> N;
  int r[N];
  int n[N];
}
parameters {
   real mu;
   real<lower=0> sigmasq;
   real b[N];
}
transformed parameters {
  real<lower=0> sigma;
  real<lower=0,upper=1> p[N];
  sigma <- sqrt(sigmasq); 
  for (i in 1:N)
    p[i] <- inv_logit(b[i]);
}
model {
  mu ~ normal(0.0, 1000.0); 
  sigmasq ~ inv_gamma(0.001, 0.001);
  b ~ normal(mu, sigma);
  r ~ binomial_logit(n, b);
}
generated quantities {
  real pop_mean;
  pop_mean <- inv_logit(mu); 
}
"""

In [3]:
stan_model = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b75445567ed4638c70765f73ec25f477 NOW.


In [4]:
init = {
    'b': [0.1,   0.1,   0.1,   0.1,   0.1,   0.1,   0.1,   0.1,   0.1,   0.1,   0.1,   0.1],
    'sigmasq': 1,
    'mu': 0
}

In [13]:
data = {
    'n': [47, 148, 119, 810, 211, 196, 148, 215, 207, 97, 256, 360],
    'r': [0, 18, 8, 46, 8, 13, 9, 31, 14, 8, 29, 24],
    'N': 12
}

In [15]:
init = [init, init, init, init] # one for each chain

In [16]:
fit = stan_model.sampling(init=init, data=data, iter=1000, chains=4)

In [17]:
fit

Inference for Stan model: anon_model_b75445567ed4638c70765f73ec25f477.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu        -2.56  3.9e-3   0.15  -2.86  -2.65  -2.55  -2.47  -2.28   1498    1.0
sigmasq    0.19  4.9e-3   0.15   0.03   0.09   0.15   0.22   0.61    997    1.0
b[0]      -2.95    0.01   0.42  -3.94  -3.19   -2.9  -2.66  -2.27   1131    1.0
b[1]      -2.19  5.4e-3   0.24  -2.64  -2.36  -2.19  -2.02  -1.71   2000    1.0
b[2]      -2.62  6.2e-3   0.28   -3.2  -2.79  -2.61  -2.44  -2.11   2000    1.0
b[3]      -2.77  3.3e-3   0.15  -3.07  -2.87  -2.77  -2.67  -2.49   2000    1.0
b[4]      -2.94  6.2e-3   0.28  -3.53  -3.12  -2.93  -2.75  -2.46   2000    1.0
b[5]      -2.62  5.2e-3   0.23  -3.13  -2.76  -2.61  -2.47  -2.19   2000    1.0
b[6]      -2.67  5.8e-3   0.26  -3.23  -2.84  -2.67   -2.5  -2.19   2000    1.0
b[7]      -1.